In [2]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  

In [7]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT * FROM vw_MetricsCompositeKeys WHERE PK_FORM = '2021-A-SAOP'
""" 
dfsaop = pd.read_sql(query,con=conn)  
dfsaop.head(2)
max_qpk =dfsaop.iloc[0].MAX_Q_PK + 52
max_qpk

25150

In [ ]:
def processdf(df):
    df = df.fillna('~') 
    #df.QTEXT = df.QTEXT.apply( lambda x: x.replace('~',',') )
    #df.QTEXT = df.QTEXT.apply( lambda x: x.replace('" ','').replace('"','') )
    return df

In [ ]:
df_new = pd.read_csv(fr'C:\Users\Tim\Documents\20-01\metrics.csv') 
df_new  = processdf(df_new)  
df_new  = df_new 

In [ ]:
sql = ''
aspx = ''  
dlist = []
for i, r in df_new[:].iterrows():
    QPK = QPK + 1 
    r.ID = str(r.ID)
    m = str(r.ID).replace('.0','.1') 
    dlist.append(r.to_dict())
    dlist.append({'ID':f'{m}','QTEXT':'Notes'})
df_new = pd.DataFrame(dlist);     
df_new["SQL"] = ""
df_new["ASPX"] = ""
df_new["QPK"] = ""
df_new[0:3]

In [ ]:
sql = ''
QPK = 23500
sord = 0
for i, r in df_new[:].iterrows():
    QPK=QPK+1 
    sord=sord+1
    df_new.loc[i, 'QPK']= str(QPK)
    df_new.loc[i, 'SQL'] = f"({QPK}, @FormName, @PK_QGroup, {sord}, N'{r.ID}', NULL, NULL, NULL, 18, 1, NULL, 0, NULL, NULL, NULL, N'{r.QTEXT.strip()}',  NULL),"
    if 'xx.xxx' in r.ID:
        sord=0
        QPK=QPK+50
df_mas[0:3]

In [ ]:
aspx = '' 
s =''
sordr = 0
for i, r in df_mas[:].iterrows(): 
    if '.0' in r.ID:
        s = open(f'C:\\_som\\_src\\PICK.aspx', 'r').read()
    else:
        s = open(f'C:\\_som\\_src\\TXT.aspx', 'r').read() 
    metric = str(r.ID).replace('.','_')
    s = s.replace('{0}',str(r.QPK)).replace('{1}',str(metric)).replace('{4}',str(metric)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTEXT)).replace('{5}','')
    s = s.replace('{2}',str(160))
    df_mas.loc[i, 'ASPX'] = s 
df_mas.to_csv(r"C:\Users\Tim\Documents\20-01\df_mas.csv", index='false') 
df_mas[0:3]

In [ ]:
sql=''
aspx=''
for i, r in df_mas[:].iterrows(): 
    sql=sql+'\n'+r.SQL
    aspx=aspx+'\n\n'+r.ASPX
    
with open(r'C:\Users\Tim\Documents\20-01\script.sql', 'w') as file:
  file.write(sql)
file.close()
with open(r'C:\Users\Tim\Documents\20-01\aspx.aspx', 'w') as file:
  file.write(aspx)
file.close()

df_mas[0:5]

In [ ]:
showrow = """\
    Protected _ShowRow_{0} As String
    Public Property ShowRow_{0}() As String
        Get
            Return _ShowRow_{0}
        End Get
        Set(ByVal value As String)
            _ShowRow_{0} = value
        End Set
    End Property
    
"""

In [ ]:
sr=''
for i,r in df_mas.iterrows():
    sr = sr + showrow.replace('{0}', r.ID.replace(".","_"))
print(sr)

In [ ]:
import nltk
stop_words = set(stopwords.words('english'))  